# Parallel Coordinate Plots

This notebook demonstrates preprocessing and plotting of the parallel coordinate plots displayed in Fig. 3c,d.

### Note

To replicate the results presented in the publication, it is essential to preprocess the complete ESM output data as described in the README or Method section of the publication.

**Following steps are included in this script:**

1. **Load Period Means**:
   - Load preprocessed data for historical and future periods across multiple models and variables.
2. **Compute BGWS, Ensemble Mean/Median and the Change**:
   - Calculate Blue-Green Water Share (BGWS) and ensemble mean changes.
3. **Subdivide BGWS Regimes and Compute Spatial Means**
4. **Create Parallel Coodinate Plots**
   - Select variables and set y-axis limits.
   - Create the plots.

In [ ]:
# ========== Import Required Libraries ==========
import sys
import dask
from dask.diagnostics import ProgressBar

In [ ]:
# ========== Configure Paths ==========
# Define the full path to the directories containing utility scripts and configurations
sys.path.append('../../src')
sys.path.append('../../src/data_handling')
sys.path.append('../../src/analysis')
sys.path.append('../../src/visualization')

# Import custom utility functions and configurations
import load_data as load_dat
import process_data as pro_dat
import compute_statistics as comp_stats
import parallel_coordinate_plots as par_coor_plot

#import data directory
from config import DATA_DIR

In [ ]:
# ========== Define Font Style ==========
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Nimbus Sans'

### Step 1: Load Period Means

In [ ]:
# Step 1.1: Define the datasets
experiments = ['historical', 'ssp370']
models = ['BCC-CSM2-MR', 'CESM2', 'CMCC-CM2-SR5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 
          'IPSL-CM6A-LR', 'MIROC-ES2L', 'MPI-ESM1-2-LR', 'NorESM2-MM', 'UKESM1-0-LL']
variables=['pr', 'tran', 'mrro', 'vpd', 'mrso', 'lai', 'wue', 'evapo', 'RX5day']

In [ ]:
# Step 1.2: Load the datasets
print("Loading period means...")
with ProgressBar():
    ds_dict = dask.compute(
        load_dat.load_period_mean(
            DATA_DIR, 'processed', experiments, models, variables
        )
    )[0]

### Step 2: Compute BGWS, Ensemble Mean/Median and the Change

In [ ]:
# Step 2.1: # Compute BGWS and ensemble mean/median for both periods
ds_dict = pro_dat.compute_bgws(ds_dict)
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'mean')
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'median')
ds_dict['ssp370'] = comp_stats.compute_ensemble_statistic(ds_dict['ssp370'], 'mean')
ds_dict['ssp370'] = comp_stats.compute_ensemble_statistic(ds_dict['ssp370'], 'median')

In [ ]:
# Step 2.2: Compute change dictionaries and ensemble mean
ds_dict_change = pro_dat.compute_change_dict(ds_dict)
ds_dict_change['ssp370-historical'] = comp_stats.compute_ensemble_statistic(ds_dict_change['ssp370-historical'], 'mean')
ds_dict_change['ssp370-historical'] = comp_stats.compute_ensemble_statistic(ds_dict_change['ssp370-historical'], 'median')

### Step 3: Subdivide BGWS Regimes and Compute Spatial Means

In [ ]:
#Step 3.1: Subdivide historical data
ds_dict_sub = {}
ds_dict_sub = pro_dat.subdivide_ds_dict(ds_dict['historical'], ds_dict['historical'], 
                                                    'historical', 'historical', 'bgws')

In [ ]:
#Step 3.2: Subdivide change data
ds_dict_change_sub = {}
ds_dict_change_sub = pro_dat.subdivide_ds_dict(ds_dict['historical'], ds_dict_change['ssp370-historical'], 
                                                    'historical', 'ssp370-historical', 'bgws')

In [ ]:
#Step 3.3: Compute weighted spatial mean for historical data
ds_dict_sub_mean = {}
ds_dict_sub_mean = comp_stats.compute_spatial_mean_with_subdivisions(ds_dict_sub)

In [ ]:
#Step 3.4: Compute weighted spatial mean for change data
ds_dict_change_sub_mean = {}
ds_dict_change_sub_mean = comp_stats.compute_spatial_mean_with_subdivisions(ds_dict_change_sub)

### Step 4: Create Parallel Coodinate Plots

In [ ]:
#Step 4.1: Select variables and set y-axis limits
selected_variables = ['bgws','RX5day', 'pr', 'mrro','tran', 'evapo', 'mrso', 'lai', 'wue', 'vpd']
yaxis_limits = {'vpd': 6.5, 'RX5day': 24, 'pr': 0.4, 'mrro': 0.4, 'tran': 0.4, 'evapo': 0.4, 'mrso': 12, 'lai': 0.9, 'wue': 2, 'bgws': 9}

In [ ]:
#Step 4.2: Create plots
par_coor_plot.create_parallel_coordinate_plots(
                                    ds_dict_change_sub_mean['ssp370-historical'], 
                                    ds_dict_sub_mean['historical'], 
                                    selected_variables, 
                                    yaxis_limits,
                                    savepath=None
)

In [ ]:
# ========= Define period, models and path ==============
data_state = 'processed'
data_product = 'CMIP6'
experiments = ['historical', 'ssp370']
models = ['BCC-CSM2-MR', 'CESM2', 'CMCC-CM2-SR5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 
          'IPSL-CM6A-LR', 'MIROC-ES2L', 'MPI-ESM1-2-LR', 'NorESM2-MM', 'UKESM1-0-LL']

variables=['tas', 'pr', 'vpd', 'evspsbl', 'evapo', 'tran', 'mrro', 'mrso', 'lai', 'gpp', 'wue', 'RX5day', 'gsl'] 
           
# Execute load function with Dask
with ProgressBar():
    ds_dict = dask.compute(load_dat.load_period_mean(BASE_DIR, data_state, data_product, experiments, models, variables, specific_months_or_seasons=None))[0]

In [ ]:
ds_dict = pro_dat.compute_bgws(ds_dict)

In [ ]:
ds_dict = pro_dat.compute_tbgw(ds_dict)

In [ ]:
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'mean')
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'median')

In [ ]:
ds_dict['ssp370'] = comp_stats.compute_ensemble_statistic(ds_dict['ssp370'], 'mean')
ds_dict['ssp370'] = comp_stats.compute_ensemble_statistic(ds_dict['ssp370'], 'median')